# Codigo para leer un archivo csv

In [ ]:
import csv
import os
import cProfile
import time
import numpy as np
import pandas as pd
import time

cwd = os.getcwd()
print('terminados los import')

#### Variables para desarrollo

In [ ]:

dirdata = 'C:\\Users\\pol\\Documents\\JupyterNotebooks\Machine Learning\\';
dirlistas = 'C:\\Users\\pol\\Documents\\JupyterNotebooks\Machine Learning\\';
datafile = 'MOCK_DATA_full.csv';
ctrl_usr_lst='ctrl_user.csv';
ctr_ip_lst='ip_ctrl.csv';

ACTION_COL='ACTIVIDAD';
RETURN_COL='RETURN_CODE';
USER_COL='ID_USER';
IP_COL='IP_ORIGEN';
EMAIL_COL='EMAIL';
TIMESTAMP_COL='FECHA_HORA';

OK_RCODE= 'NAZ0000';
LOGIN_CODE='login';
MODIF_USER_DATA_CODE='cambiodatos'
print('cargadas variables desarrollo')

#### Variables para testing

In [ ]:
dirdata = 'C:\\Users\\pnfernandez\\Desktop\\cosos\\MATERIAL_PROD\\OPERATIVO\\INCIDENTES\\incidente_produccion_XXX\\casos\\20180910\\'
dirlistas = 'C:\\Users\\pnfernandez\\Desktop\\cosos\\MATERIAL_PROD\\OPERATIVO\\INCIDENTES\\incidente_produccion_XXX\\consultasSeguridad\\'
#datafile = 'export_audit_login_hist_20180815-0909.csv';
datafile = 'export_audit_1446.csv';
ctrl_usr_lst='lista_usuarios.csv';
ctr_ip_lst='lista_ip.csv';

ACTION_COL='ID_ACTIVITY';
RETURN_COL='RETURN_CODE';
USER_COL='ID_USER';
IP_COL='CLIENT_IP';
EMAIL_COL='EMAIL';
TIMESTAMP_COL='DATE_TIME';

OK_RCODE= 'NAZ0000';
LOGIN_CODE='client.login';
MODIF_USER_DATA_CODE='cambiodatos'
print('cargadas variables testing')

## Control: Ver ambiente en curso

In [ ]:
print ('directorio actual: ' + cwd);
print ('directorio de trabajo: '+dirdata);
print ('archivo de datos: '+datafile);
print ('lista de usuarios controlados: '+ctrl_usr_lst);
print ('lista de ip controladas: ' +ctr_ip_lst);

#print('cargo datos de archivo');
#indexes =['IP_ORIGEN', 'ID_USER'];
#data= pd.read_csv(dirname+filename1, index_col=indexes)

#### Ejecucion: Cargar lista de usuarios controlados

In [ ]:
start_time = time.time()
# your script
ctrl_usr=pd.read_csv(dirlistas+ctrl_usr_lst);
print('lista de usuarios cargada');

elapsed_time = time.time() - start_time
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

#### Ejecucion: Cargar lista de ips controladas

In [ ]:
start_time = time.time();
ctrl_ip=pd.read_csv(dirlistas+ctr_ip_lst);
print('lista de ips cargada');

elapsed_time = time.time() - start_time;
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

#### Ejecucion: Cargar auditoria

In [ ]:
start_time = time.time();
data= pd.read_csv(dirdata+datafile);
print('auditoria cargada');

elapsed_time = time.time() - start_time;
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

#### Control: cabezal de datos cargados

In [ ]:
ALL = 1;# Poner 1 para todos, 0 para SOLO LOS QUE TIENEN RETURN_COL = OK
data[(ALL) | (data[RETURN_COL]==OK_RCODE)].head(5) 

#### Control: cabezal de lista de usuarios controlados

In [ ]:
ctrl_usr.head(10)

#### Control: cabezal de lista de ip controladas

In [ ]:
ctrl_ip.head(10)

## Reporte: detalle de auditoria

In [ ]:
ALL = 1; #poner 1 para LISTAR TODOS LOS USUARIOS, 0 para filtrar por usuario especifico
ALERT_USER = 25441328;# indicar el usuario especifico
ALL_IP=0;
ALERT_IP = '180.184.197.155';

data[(ALL_IP |(data[IP_COL]==ALERT_IP))& \
    (ALL) | (data[USER_COL].round() == ALERT_USER)]

## Reporte: join auditoria con lista de usuarios controlados

In [ ]:
ALL = 0; #poner 1 para LISTAR TODOS LOS USUARIOS, 0 para filtrar por usuario especifico
ALERT_USER = 43896121;# indicar el usuario especifico
data[(ALL) | \
     (data[USER_COL].round() == ALERT_USER)].\
        join(ctrl_usr.set_index(USER_COL), \
             on=USER_COL, how='inner')

## Reporte: Filtrar auditoría por email

In [ ]:
ALL = 1; #poner 1 para LISTAR TODOS LOS USUARIOS, 0 para filtrar por usuario especifico
ALERT_EMAIL = 'jkier0@patch.com';
data[(ALL) | \
     (data['EMAIL'] == ALERT_EMAIL)].\
        join(ctrl_usr.set_index(USER_COL), \
             on=USER_COL, how='inner')

## Reporte: join auditoria con lista de IP controlada

In [ ]:
data.join(ctrl_ip.set_index(IP_COL), on=IP_COL, how='inner')

## Reporte: join de login correctos con lista de ip controladas

In [ ]:
ALL = 1; #poner 1 para LISTAR TODOS LOS 
ALERT_USER = 123456;# indicar el usuario especifico
data[(data[ACTION_COL]==LOGIN_CODE) & \
     (data[RETURN_COL]==OK_RCODE)& \
    (ALL |(data[USER_COL].round()==ALERT_USER))].\
            join(ctrl_ip.set_index(IP_COL),\
              on=IP_COL, how='inner')


## Reporte: join de login INCORRECTOS con lista de usuarios controlados
Lista todas las tuplas de login que hayan sido incorrectas, tomando como base la lista de usuarios controlados.

Usar las variables ALERT_USER y ALL para filtrar por un usuario específico

In [ ]:
print('join de datos con lista de ip')
ALL = 1; #poner 1 para LISTAR TODOS LOS 
ALERT_USER = 43896121;# indicar el usuario especifico
data[(data[ACTION_COL]==LOGIN_CODE) & \
     (data[RETURN_COL]!=OK_RCODE)& \
    (ALL |(data[USER_COL].round()==ALERT_USER))].\
            join(ctrl_usr.set_index(USER_COL),\
              on=USER_COL, how='inner')

## Reporte: lista de usuarios controlados e IP que tuvieron login INCORRECTOS
Lista los usuarios controlados y sus IPs que tuvieron intento de login incorrectos

In [ ]:
print('join de datos con lista de ip\n')
ALL = 1; #poner 1 para LISTAR TODOS LOS 
ALERT_USER = 43896121;# indicar el usuario especifico
ctrlusr_loginerr_cant=data[(data[ACTION_COL]==LOGIN_CODE) & \
     (data[RETURN_COL]!=OK_RCODE)& \
    (ALL |(data[USER_COL].round()==ALERT_USER))].\
            join(ctrl_usr.set_index(USER_COL),\
              on=USER_COL, how='inner')\
                .drop_duplicates([IP_COL,USER_COL])\
                .groupby([USER_COL, IP_COL]).size();

print('\nlista de usuarios y cantidad de IP');
print(ctrlusr_loginerr_cant.groupby([USER_COL]).size());
print('\ndetalle de   usuarios e IP');
print(ctrlusr_loginerr_cant);



## Reporte: lista de IP en join de login CORRECTOS con lista de usuarios controlados

In [ ]:
print('join de datos con lista de ip\n')
ALL_USER =1; #poner 1 para LISTAR TODOS LOS 
ALERT_USER = 1000005;# indicar el usuario especifico
ALL_IP=1;
ALERT_IP = '10.234.33.45';
ip_loginok_cant =data[(data[ACTION_COL]==LOGIN_CODE) & \
     (data[RETURN_COL]==OK_RCODE)& \
    (ALL_IP |(data[IP_COL]==ALERT_IP))& \
    (ALL_USER |(data[USER_COL]==ALERT_USER))].\
            join(ctrl_usr.set_index(USER_COL),\
              on=USER_COL, how='inner')\
                .drop_duplicates([IP_COL,USER_COL])\
                .groupby([IP_COL, USER_COL]).size();
ip_loginok_cant;
print('\nlista de usuarios y cantidad de IP');
print(ip_loginok_cant.groupby([IP_COL]).size());
print('\ndetalle de   usuarios e IP');
print(ip_loginok_cant);

## lista de IP en join de login INCORRECTOS con lista de usuarios controlados (edicion)

In [ ]:
print('join de datos con lista de ip\n')
ALL =1; #poner 1 para LISTAR TODOS LOS 
ALERT_USER = 1000005;# indicar el usuario especifico
ip_loginerr_cant=data[(data[ACTION_COL]==LOGIN_CODE) & \
     (data[RETURN_COL]!=OK_RCODE)& \
    (ALL |(data[USER_COL]==ALERT_USER))].\
            join(ctrl_usr.set_index(USER_COL),\
              on=USER_COL, how='inner')\
                .drop_duplicates([IP_COL,USER_COL])\
                .groupby([IP_COL, USER_COL]).size();
ip_loginerr_cant;
print('\nlista de usuarios y cantidad de IP');
print(ip_loginerr_cant.groupby([IP_COL]).size());
print('\ndetalle de   usuarios e IP');
print(ip_loginerr_cant);

## lista de usuarios distintos joineados con ip_controlada

In [ ]:
data.join(ctrl_ip.set_index(IP_COL), on=IP_COL, how='inner').drop_duplicates([USER_COL])[USER_COL]

## Actividad de usuarios con ip duplicada (EDICION)

In [ ]:
data.join(ctrl_ip.set_index(IP_COL), on=IP_COL, how='inner').drop_duplicates([USER_COL])

In [ ]:
data.sort_values(by=[USER_COL,TIMESTAMP_COL])

In [ ]:
#data.loc[data['RETURN_CODE']== OK_RCODE]
cambiodatos=data[USER_COL].loc[data[ACTION_COL] == MODIF_USER_DATA_COL]
print(cambiodatos)

In [ ]:
data.groupby(USER_COL)[TIMESTAMP_COL].groups

In [ ]:
print('creo lista de ip-usuario');
ip_user_list = data[[IP_COL, USER_COL, RETURN_COL]].drop_duplicates([IP_COL, USER_COL,RETURN_COL]);#elimina duplicados por columnas


In [ ]:
ip_user_list
#ip_user_list.head()

In [ ]:
#lista las tuplas originales que tienen al usuario que se repite mas de 'rep'
rep = 1
g = data.drop_duplicates([IP_COL, USER_COL]).groupby([IP_COL]) #agrupa por usuario e ip


In [ ]:
#g.filter(lambda x: len(x) > rep)
g.head(100) #muestra las tuplas contenidas en los grupos

In [ ]:
#g.filter(lambda x: len(x) > rep)
g.groups

## lista las tuplas originales que tienen al usuario que se repite mas de 'rep' (EDICION)

In [ ]:

rep = 1
g = data.drop_duplicates([IP_COL, USER_COL]).groupby([IP_COL]) #agrupa por usuario e ip
g.filter(lambda x: len(x) > rep)

In [ ]:
#lista las tuplas originales que tienen al usuario que se repite mas de 'rep'
rep = 1
g = data.drop_duplicates([IP_COL, USER_COL]).groupby([IP_COL]) #agrupa por usuario e ip
g.filter(lambda x: len(x) > rep)

In [ ]:
ip_gb_user = ip_user_list.groupby([IP_COL]).size(); #agrupa por usuario e ip
ip_gb_user
#ip_gb_user.head()

In [ ]:
email_user_list = data[[EMAIL_COL, USER_COL]].drop_duplicates([EMAIL_COL, USER_COL]);#elimina duplicados por columnas
email_user_list
#ip_user_list.head()

In [ ]:
email_gb_user = email_user_list.groupby([EMAIL_COL]).size(); #agrupa por usuario e ip
email_gb_user
#ip_gb_user.head()

In [ ]:
#agrupa por ip y usuario desde la lista ya filtrada. no repite usuario-ip
ip_user_list.groupby([IP_COL, USER_COL]).size()

In [ ]:
#agrupa por ip y usuario 
#(repetido el group by para que no cuente duplicado al mismo ip-usuario 
# al trabajar desde el dataframe original con duplicados)

data.groupby([IP_COL, USER_COL]).size().groupby([IP_COL, USER_COL]).size() 

In [ ]:
#lista las tuplas originales que tienen al usuario que se repite en mas ip que 'rep'
rep = 1
g1 = data.drop_duplicates([IP_COL, USER_COL]).groupby([USER_COL]) #agrupa por usuario e ip

g1.filter(lambda x: len(x) > rep)


In [ ]:
g2 = ip_user_list.groupby([USER_COL]) #agrupa por usuario e ip
g2.filter(lambda x: len(x) > rep)

## lista las tuplas originales que tienen a la ip que se repite en mas ip que 'rep' (EDICION)

In [ ]:
#
rep = 1
g = data.drop_duplicates([IP_COL, USER_COL]).groupby([IP_COL]) #agrupa por usuario e ip
g.filter(lambda x: len(x) > rep)

In [ ]:
data.groupby([IP_COL]).size().head()

In [ ]:
g = data.groupby([IP_COL, USER_COL])
#g = data.groupby(['ip','usuario']).size()
g.filter(lambda x: len(x) > 1)

In [ ]:
data.groupby([IP_COL, USER_COL]).size().head() #agrupa por ip y usuario

In [ ]:
data.dtypes